<a href="https://colab.research.google.com/github/JedWojtowicz/MAD_jw/blob/main/JW_lab_11_PD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 11. Inne przykłady wykresów `plotly` oraz analiza głównych składowych

## 1. Kilka innych przykładów wykresów z biblioteką `plotly`

In [81]:
# zainstaluj, jeżeli brakuje biblioteki
import plotly as py
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets
from google.colab import output
output.enable_custom_widget_manager()
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

In [82]:
!pip install scikit-learn

In [83]:
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, KernelPCA

**Zadanie 1**

Wykorzystując dowolny zbiór danych ze zbioru EUROSTAT, wyświetl jedną z cech numerycznych w postaci wykresu z mapą z przykładu w tym labie. Musisz dokonać mapowania nazw krajów, gdyż w parametrach wykresu oraz danych EUROSTAT te sktóry się trochę różnią. Link do pliku, który to ułatwi jest w komentarzu w kodzie przykładu wykresu z mapą.

In [84]:
!pip install eurostat
import eurostat

In [85]:
import json
import requests

In [86]:
data = eurostat.get_data_df('ilc_di03')
data = data.rename(columns={r'geo\TIME_PERIOD': 'geo'})
data

,freq,age,sex,indic_il,unit,geo,1995,1996,1997,1998,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,A,TOTAL,F,MED_E,EUR,AL,NaN,NaN,NaN,NaN,...,NaN,NaN,1747.0,1972.0,2195.0,2482.0,2574.0,NaN,NaN,NaN
1,A,TOTAL,F,MED_E,EUR,AT,13394.0,14277.0,13841.0,13677.0,...,22741.0,23202.0,24433.0,24675.0,25241.0,26348.0,27048.0,27646.0,30988.0,32358.0
2,A,TOTAL,F,MED_E,EUR,BE,13085.0,13735.0,13707.0,13669.0,...,21012.0,21550.0,21976.0,23135.0,24013.0,25135.0,25356.0,26826.0,28454.0,29884.0
3,A,TOTAL,F,MED_E,EUR,BG,NaN,NaN,NaN,NaN,...,3236.0,3043.0,3480.0,3474.0,4059.0,4424.0,5012.0,5117.0,6271.0,7435.0
4,A,TOTAL,F,MED_E,EUR,CH,NaN,NaN,NaN,NaN,...,38361.0,43091.0,42640.0,42253.0,40146.0,41911.0,43917.0,44520.0,48394.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17641,A,Y_LT75,T,MEI_E,PPS,SI,NaN,NaN,NaN,NaN,...,16327.0,16491.0,16445.0,17005.0,18252.0,18422.0,19109.0,20363.0,21956.0,23533.0
17642,A,Y_LT75,T,MEI_E,PPS,SK,NaN,NaN,NaN,NaN,...,10824.0,11194.0,10092.0,10342.0,11121.0,10509.0,9816.0,10172.0,10772.0,11668.0
17643,A,Y_LT75,T,MEI_E,PPS,TR,NaN,NaN,NaN,NaN,...,7810.0,9056.0,9217.0,9112.0,9244.0,9225.0,8604.0,9983.0,10456.0,14220.0
17644,A,Y_LT75,T,MEI_E,PPS,UK,13629.0,13927.0,14560.0,15433.0,...,21555.0,20494.0,22081.0,22439.0,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
iso3_map = requests.get("https://country.io/iso3.json").json()
year_to_plot = '2022'

In [116]:
df_filtered = data[(data["indic_il"] == "MED_E") & (data['sex'] == 'T') & (data['age'] == 'TOTAL')]
unit = df_filtered['unit']
geo = df_filtered['geo']
year = df_filtered[year_to_plot]
df = pd.DataFrame({'geo': geo, 'unit': unit, year_to_plot: year})
df["geo"] = df_filtered["geo"].map(iso3_map)
df.dropna(inplace=True)
df = df[(df['unit'] == "EUR") | (df['unit'] == "PPS")]
df

,geo,unit,2022
517,AUT,EUR,27844.0
518,BEL,EUR,27314.0
519,BGR,EUR,5378.0
520,CHE,EUR,45405.0
521,CYP,EUR,17856.0
...,...,...,...
638,SRB,PPS,7411.0
639,SWE,PPS,20608.0
640,SVN,PPS,18792.0
641,SVK,PPS,9826.0


In [117]:
values = df[year_to_plot]
hover_texts = df["geo"] + " - " + df[year_to_plot].astype(str)
hover_texts

,0
517,AUT - 27844.0
518,BEL - 27314.0
519,BGR - 5378.0
520,CHE - 45405.0
521,CYP - 17856.0
...,...
638,SRB - 7411.0
639,SWE - 20608.0
640,SVN - 18792.0
641,SVK - 9826.0


In [118]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot


init_notebook_mode(connected = True)

data_to_plot = dict(type = 'choropleth',

            locations = df['geo'],


            locationmode = 'ISO-3',


            colorscale = 'Reds',

            text =hover_texts,

            z = values,
            zmin = values.min(),
            zmax = values.max(),

            colorbar = {'title': ('Mediana Dochodu (PPS) w roku ' + year_to_plot)})

layout = dict(geo ={'scope': 'europe'})

choromap = go.Figure(data = [data_to_plot], layout = layout)

#sposób wyświetlenia mapy na google colab
import plotly.io as pio
pio.renderers.default = 'colab'

#iplot(choromap)

choromap.show()

**Zadanie 2**

Wykorzystując zbiór wine z biblioteki sklearn (load_wine(), podobnie jak ze zbiore iris) dokonaj analizy PCA tak aby:
* zachować minimum 99% wariancji
* zachować minimum 95% wariancji
* zachować minimum 90% wariancji

Ile cech pozostało w tych zbiorach?

In [91]:
from sklearn.datasets import load_wine

wine = load_wine()
feature_names = wine.feature_names
feature_names

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [92]:
wine_df = pd.DataFrame(np.hstack([wine.data, np.expand_dims(wine.target, axis=1)]), columns=wine.feature_names + ['target'])

In [93]:
print(wine.data.shape)
# 99%
pca = PCA(n_components=.99)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_1 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
print(reduced_1.shape)
# 95%
pca = PCA(n_components=.95)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_2 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
print(reduced_2.shape)
# 90%
pca = PCA(n_components=.90)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_3 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
print(reduced_3.shape)

(178, 13)
(178, 1)
(178, 1)
(178, 1)


**Zadanie 3**  

Wykorzystując przykład w labie wygeneruj wykres korelacji wartości cech wględem klasy decyzyjnej (target). Czy cechy, które PCA odrzuciło faktycznie były wskazane jako najlepsi kandydaci pod względem separowalności liniowej?

In [94]:
pca = PCA(n_components=13)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_1 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
pca.explained_variance_ratio_.round(3)

array([0.998, 0.002, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   ])

**Zadanie 4**  

Wykorzystując przykład w skrypcie z wykładu numer 11 (https://github.com/agazbrzezny/MAD_SWPS_2025/blob/master/wyklad_10/redukcja_wielowym.ipynb) pokazujący użycie `SelectFromModel` (punkt 4 wykładu) wykonaj tę samą sekwencję czynności z użyciem klasyfikatora RandomForest do selekcji cech. Po przetrenowaniu modelu drzewa stwórz 3 zbiory cech z progiem jak w zadaniu 2 sumując `feature importance`. Czy zostały wybrane te same cechy co w zadaniu 2?

In [95]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [96]:
X, y = load_wine(return_X_y=True)

In [97]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# 2. Budowa pipeline
pipeline = Pipeline([
    # Standaryzacja: cechy o różnych skalach zostaną wyrównane
    ('scaler', StandardScaler()),

    # Selekcja cech:
    # - RandomForestClassifier oblicza feature_importances_
    # - SelectFromModel odrzuca cechy poniżej mediany importance
    ('feature_sel', SelectFromModel(
        RandomForestClassifier(n_estimators=50, random_state=42),
        threshold='median'
    )),

    # Klasyfikator końcowy: regresja logistyczna
    ('clf', LogisticRegression(max_iter=1000))
])

# 3. Trenowanie całego pipeline
pipeline.fit(X_train, y_train)

# 4. Sprawdzenie liczby cech po selekcji
n_selected = pipeline.named_steps['feature_sel'] \
                    .transform(X_train).shape[1]
print("Liczba cech po selekcji:", n_selected)

# 5. Ocena dokładności na zbiorze walidacyjnym
accuracy = pipeline.score(X_val, y_val)
print("Accuracy na zbiorze walidacyjnym:", accuracy)

Liczba cech po selekcji: 7
Accuracy na zbiorze walidacyjnym: 1.0


In [98]:
feature_names = wine.feature_names  # np. ["sepal length", "sepal width", ...]

# 1. Pobierz krok selekcji z pipeline
selector = pipeline.named_steps['feature_sel']

# 2. Uzyskaj maskę boolean
mask = selector.get_support()
# mask to tablica np. [True, False, True, True] – True = cecha zachowana

# 3. Wypisz nazwy wybranych cech
selected_features = [name for name, keep in zip(feature_names, mask) if keep]
print("Wybrane cechy:", selected_features)

# 4. (Opcjonalnie) Indeksy wybranych cech
import numpy as np
selected_indices = np.where(mask)[0]
print("Indeksy wybranych cech:", selected_indices)

Wybrane cechy: ['alcohol', 'total_phenols', 'flavanoids', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
Indeksy wybranych cech: [ 0  5  6  9 10 11 12]
